# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   4 months ago   1.73GB


If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:9f6ad537c5132bcce57f7a0a20e317228d382c3cd61edae14650eec68b2b345c
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https

Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
hello-world                    latest    d1165f221234   3 months ago   13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   4 months ago   1.73GB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED        STATUS        PORTS     NAMES
7f043ca3ee95   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   18 hours ago   Up 18 hours             proxy-agent


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [5]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                     PORTS     NAMES
c6af7bbc8202   hello-world                    "/hello"                 10 seconds ago   Exited (0) 8 seconds ago             compassionate_almeida
7f043ca3ee95   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   18 hours ago     Up 18 hours                          proxy-agent


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application.

**Exercise**

Open the text file called `intro.docker` in the `dockerfiles` folder and complete the TODO there. 

Your dockerfile should have the following steps

 1. use `FROM` to inherit an official Node runtime as the parent image; e.g. node:6
 2. use `WORKDIR` to seet the working directory to /app
 3. use `ADD` to copy the current directory to the container at /app
 4. use `EXPOSE` to make the containers port 80 available to the outside world
 5. use `CMD` to run the command `node ./src/app.js`

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

**Exercise**

Use `docker build` to build the docker image at `dockerfiles/intro.docker`. Tag the image `node-app:0.1`. 

In [19]:
%%bash
docker build -t node-app:0.1 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  93.18kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> bfd2e0787a5f
Step 3/5 : ADD src/* /app
 ---> Using cache
 ---> c3ebcb7cc20e
Step 4/5 : EXPOSE 80
 ---> Running in 7e4639ee28e8
Removing intermediate container 7e4639ee28e8
 ---> 79504cb41ecb
Step 5/5 : CMD node app.js
 ---> Running in e500677fe360
Removing intermediate container e500677fe360
 ---> 2c034c8bfa99
Successfully built 2c034c8bfa99
Successfully tagged node-app:0.1


Let's check that the image has been created correctly. 

In [20]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED                  SIZE
node-app                       0.1       2c034c8bfa99   Less than a second ago   884MB
<none>                         <none>    76a2be01f545   2 minutes ago            884MB
<none>                         <none>    c919e16a1555   5 minutes ago            884MB
<none>                         <none>    74ae2c4ac7dd   7 minutes ago            884MB
hello-world                    latest    d1165f221234   3 months ago             13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   4 months ago             1.73GB
node                           6         ab290b853066   2 years ago              884MB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [21]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED              STATUS                            PORTS     NAMES
a96415c50151   76a2be01f545                   "/bin/bash"              43 seconds ago       Up 42 seconds                     80/tcp    stoic_goldberg
fdd177b54116   76a2be01f545                   "/bin/sh -c ./src/ap…"   About a minute ago   Exited (127) About a minute ago             my-app
7f043ca3ee95   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   18 hours ago         Up 18 hours                                 proxy-agent


**Exercise**

Use `docker run` to run the container you just build called `node-app:0.1`. Assign the host port `4000` to port `80` and assign it the name `my-app`.

In [23]:
%%bash
docker run --name my-app -p 4000:80 node-app:0.1

Process is terminated.


To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

**Exercise**

Modify your command above with `-d` flag to run `my-app` in the background.

In [27]:
%%bash
docker run -d --name my-app -p 4000:80 node-app:0.1

4cc675f1ce205a3636c8e26321d72ef609f86c21b65e399833a7992158a6d495


Your container is now running in the background. You can check the status of your running container using `docker ps`

In [28]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED         STATUS        PORTS                  NAMES
4cc675f1ce20   node-app:0.1                   "/bin/sh -c 'node ap…"   2 seconds ago   Up 1 second   0.0.0.0:4000->80/tcp   my-app
7f043ca3ee95   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   18 hours ago    Up 18 hours                          proxy-agent


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [30]:
# Note, your container id will be different
!docker logs 4cc675f1ce20   

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

**Exercise**

After modifying the `app.js` file, use `docker build` to build a new container called `node-app:0.2` from the same docker file. 

In [33]:
%%bash
docker build -t node-app:0.2 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  88.06kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> bfd2e0787a5f
Step 3/5 : ADD src/* /app/
 ---> 1daa53ff3a82
Step 4/5 : EXPOSE 80
 ---> Running in edbc3fd3d2fd
Removing intermediate container edbc3fd3d2fd
 ---> 346369d972d4
Step 5/5 : CMD node app.js
 ---> Running in e5b5a6656172
Removing intermediate container e5b5a6656172
 ---> f8e2bfa274cc
Successfully built f8e2bfa274cc
Successfully tagged node-app:0.2


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

**Exercise**

Run this new container in the background using a different port and with the name `my-app-2`.

In [34]:
%%bash
docker run -d --name my-app-2 -p 5000:80 node-app:0.2

a431c5190918aa55b1a71d9e0ae8b7cd967db6ed180e15e111494460343dc18a


You can check that both container are running using `docker ps`.

In [35]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED         STATUS         PORTS                  NAMES
a431c5190918   node-app:0.2                   "/bin/sh -c 'node ap…"   4 seconds ago   Up 2 seconds   0.0.0.0:5000->80/tcp   my-app-2
4cc675f1ce20   node-app:0.1                   "/bin/sh -c 'node ap…"   2 minutes ago   Up 2 minutes   0.0.0.0:4000->80/tcp   my-app
7f043ca3ee95   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   18 hours ago    Up 18 hours                           proxy-agent


And let's test boht containers using `curl` as before:

In [36]:
!curl http://localhost:5000

Nice Docker!


In [37]:
!curl http://localhost:4000

Hello Wolrd!


Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [38]:
!gcloud config list

[core]
account = 147678809820-compute@developer.gserviceaccount.com
disable_usage_reporting = True
project = qwiklabs-gcp-00-0db9b1bc58c6

Your active configuration is: [default]


In [39]:
import os

PROJECT_ID = "qwiklabs-gcp-00-0db9b1bc58c6" # REPLACE WITH YOUR PROJECT NAME

os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [40]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED              SIZE
node-app                       0.2       f8e2bfa274cc   About a minute ago   884MB
node-app                       0.1       2c034c8bfa99   4 minutes ago        884MB
hello-world                    latest    d1165f221234   3 months ago         13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   4 months ago         1.73GB
node                           6         ab290b853066   2 years ago          884MB


**Exercise**

Tag the `node-app:0.2` image with a new image name conforming to the naming convention `gcr.io/[project-id]/[image]:[tag]`. Keep the image and tag names the same.

In [41]:
%%bash
docker tag node-app:0.2 gcr.io/${PROJECT_ID}/node-app:0.2

Now when we list our docker images we should see this newly tagged repository.

In [42]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED              SIZE
node-app                                       0.2       f8e2bfa274cc   About a minute ago   884MB
gcr.io/qwiklabs-gcp-00-0db9b1bc58c6/node-app   0.2       f8e2bfa274cc   About a minute ago   884MB
node-app                                       0.1       2c034c8bfa99   5 minutes ago        884MB
hello-world                                    latest    d1165f221234   3 months ago         13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   4 months ago         1.73GB
node                                           6         ab290b853066   2 years ago          884MB


Next, let's push this image to gcr.

**Exercise**

Push this new image to the gcr.

In [43]:
%%bash
docker push gcr.io/${PROJECT_ID}/node-app:0.2

The push refers to repository [gcr.io/qwiklabs-gcp-00-0db9b1bc58c6/node-app]
c7bda980a817: Preparing
405d86aa468a: Preparing
f39151891503: Preparing
f1965d3c206f: Preparing
a27518e43e49: Preparing
910d7fd9e23e: Preparing
4230ff7f2288: Preparing
2c719774c1e1: Preparing
ec62f19bb3aa: Preparing
f94641f1fe1f: Preparing
910d7fd9e23e: Waiting
4230ff7f2288: Waiting
2c719774c1e1: Waiting
f94641f1fe1f: Waiting
ec62f19bb3aa: Waiting
a27518e43e49: Layer already exists
910d7fd9e23e: Layer already exists
4230ff7f2288: Layer already exists
2c719774c1e1: Layer already exists
ec62f19bb3aa: Layer already exists
c7bda980a817: Pushed
f94641f1fe1f: Layer already exists
405d86aa468a: Pushed
f39151891503: Pushed
f1965d3c206f: Pushed
0.2: digest: sha256:bfaed00f7401fea188e899ec53f73810e83463f91aa581d3a0054627c5b82c76 size: 2422


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [44]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-gcp-00-0db9b1bc58c6/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [45]:
!docker stop my-app && docker rm my-app

my-app
my-app


In [46]:
!docker stop my-app-2 && docker rm my-app-2

my-app-2
my-app-2


Now remove the docker images you've created above using `docker rmi`.

In [47]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED         SIZE
node-app                                       0.2       f8e2bfa274cc   2 minutes ago   884MB
gcr.io/qwiklabs-gcp-00-0db9b1bc58c6/node-app   0.2       f8e2bfa274cc   2 minutes ago   884MB
node-app                                       0.1       2c034c8bfa99   6 minutes ago   884MB
hello-world                                    latest    d1165f221234   3 months ago    13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   4 months ago    1.73GB
node                                           6         ab290b853066   2 years ago     884MB


In [48]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:6 
docker rmi -f hello-world:latest

Untagged: node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-00-0db9b1bc58c6/node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-00-0db9b1bc58c6/node-app@sha256:bfaed00f7401fea188e899ec53f73810e83463f91aa581d3a0054627c5b82c76
Deleted: sha256:f8e2bfa274cc9009fc2c262a905a633954dc372a34f8703d4352e76f24ce1a63
Deleted: sha256:346369d972d4690b93c4e8d606daaf234626bc9fb43ea2ed124eef7aee1d01f5
Deleted: sha256:1daa53ff3a82ed749cfa6ecd829a6d3719be24cfe012091493ac132b7c5be429
Deleted: sha256:0c69a0e66d1e3b16db30d8f869783a0080ca3ccf89bc7159315c0f5e90e24a0a
Untagged: node-app:0.1
Deleted: sha256:2c034c8bfa9991491454264772a7f9129261bfe853d54ad80a5bd626948bd315
Deleted: sha256:79504cb41ecb9b4c43641fe439a72919720f94b7e46465ecdad2807ea6da5c9d
Deleted: sha256:c3ebcb7cc20e738632adb407a679c30584adcd02d8b8bc8dd129cb4d37422e2b
Deleted: sha256:1fd69a5d154b0bfe777ccd7cabc303791b5046bed1e5c2105eb27f39cc2145bc
Deleted: sha256:bfd2e0787a5ff2df9c881af2a29718010bcee9453dcd28d62446ff0374ed9e89
Deleted: sha256:3a9be440fb10fce07285d98

Confirm all images are removed with `docker images`.

In [49]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   4 months ago   1.73GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [50]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-gcp-00-0db9b1bc58c6/node-app
c5e155d5a1d1: Pulling fs layer
221d80d00ae9: Pulling fs layer
4250b3117dca: Pulling fs layer
3b7ca19181b2: Pulling fs layer
425d7b2a5bcc: Pulling fs layer
69df12c70287: Pulling fs layer
ea2f5386a42d: Pulling fs layer
d421d2b3c5eb: Pulling fs layer
336a8fdcf9a0: Pulling fs layer
3b7ca19181b2: Waiting
425d7b2a5bcc: Waiting
69df12c70287: Waiting
bd69e83bc719: Pulling fs layer
ea2f5386a42d: Waiting
d421d2b3c5eb: Waiting
336a8fdcf9a0: Waiting
bd69e83bc719: Waiting
4250b3117dca: Download complete
221d80d00ae9: Verifying Checksum
221d80d00ae9: Download complete
3b7ca19181b2: Verifying Checksum
3b7ca19181b2: Download complete
c5e155d5a1d1: Verifying Checksum
c5e155d5a1d1: Download complete
69df12c70287: Download complete
d421d2b3c5eb: Verifying Checksum
d421d2b3c5eb: Download complete
ea2f5386a42d: Verifying Checksum
ea2f5386a42d: Download complete
336a8fdcf9a0: Verifying Checksum
336a8fdcf9a0: Download complete
bd69e83bc719: Verifying Ch

You can check that it's running as expected using before:

In [51]:
!curl http://localhost:4000

Nice Docker!


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.